In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np


In [5]:
from pathlib import Path
paths = ['respondent_1','respondent_2','respondent_3','respondent_4']
file_paths = {}
for name in paths:
    folder = Path(name)
    files = [name + '/' + f.name for f in folder.iterdir() if f.is_file()]
    for i, file in enumerate(sorted(files, key=lambda x: x[-24:-5])):
        # потрошим файл
        s = file.split('_')
        timestamp = ''
        if s[-1] == 'test.csv':
            timestamp = pd.to_datetime(s[-3] + ' ' + s[-2], format='%Y-%m-%d %H-%M-%S') + pd.Timedelta(hours=3)
        else:
            timestamp = pd.to_datetime(s[-4] + ' ' + s[-3], format='%Y-%m-%d %H-%M-%S')
        if timestamp < pd.to_datetime('2024-12-12 00:00:00') and name == 'respondent_4':
            continue
        file_paths[(name, timestamp)] = file

file_paths

{('respondent_1',
  Timestamp('2024-12-11 17:20:52')): 'respondent_1/Юля_2024-12-11_14-20-52_test.csv',
 ('respondent_1',
  Timestamp('2024-12-12 13:14:12')): 'respondent_1/Юля_2024-12-12_10-14-12_test.csv',
 ('respondent_1',
  Timestamp('2024-12-12 21:07:06')): 'respondent_1/Юля_2024-12-12_18-07-06_test.csv',
 ('respondent_1',
  Timestamp('2024-12-13 08:20:01')): 'respondent_1/Юля_2024-12-13_05-20-01_test.csv',
 ('respondent_1',
  Timestamp('2024-12-13 20:13:02')): 'respondent_1/Юля_2024-12-13_17-13-02_test.csv',
 ('respondent_1',
  Timestamp('2024-12-14 11:56:28')): 'respondent_1/Юля_2024-12-14_08-56-28_test.csv',
 ('respondent_1',
  Timestamp('2024-12-14 20:53:06')): 'respondent_1/Юля_2024-12-14_17-53-06_test.csv',
 ('respondent_1',
  Timestamp('2024-12-15 12:06:37')): 'respondent_1/Юля_2024-12-15_09-06-37_test.csv',
 ('respondent_1',
  Timestamp('2024-12-15 22:12:43')): 'respondent_1/Юля_2024-12-15_19-12-43_test.csv',
 ('respondent_2',
  Timestamp('2024-12-11 16:09:52')): 'responde

In [7]:
import pandas as pd
import json

# Initialize empty DataFrames
answersLog_df = pd.DataFrame()
typingAnswersLog_df = pd.DataFrame()
testAnswersLog_df = pd.DataFrame()
stressTestLog_df = pd.DataFrame()
pageMetaData_df = pd.DataFrame()
mouseTrack_df = pd.DataFrame()

# Read and concatenate data from all files
for ((name, date), file_path) in file_paths.items():
    temp_df = pd.read_csv(file_path)
    temp_df["id"] = name
    temp_df['datetime'] = date
    # Извлекаем день недели в новый столбец
    temp_df['weekday'] = temp_df['datetime'].dt.day_name()
    temp_df['weekday_num'] = temp_df['datetime'].dt.dayofweek
    temp_df["hour"] = temp_df['datetime'].dt.hour
    temp_df["hour"] = temp_df["hour"].apply(lambda h: h + 24 if h < 5 else h)

    temp_df['date'] = temp_df['datetime'].dt.date
    temp_df['date'] =  temp_df[['datetime', 'hour', 'date']].apply(lambda row: (row['datetime'] - pd.Timedelta(days=1)).date() if row['hour'] > 23 else row['date'], axis=1)

    answersLog_df = pd.concat([answersLog_df, temp_df], ignore_index=True)
answersLog_df.head()

,#,num1,num2,operation,right_ans,user_ans,start_time,end_time,correct_flg,id,datetime,weekday,weekday_num,hour,date
0,0,7,3,sd_abs_sub,4,4,751245.656,751247.703,1,respondent_1,2024-12-11 17:20:52,Wednesday,2,17,2024-12-11
1,1,6,1,sd_abs_sub,5,5,751247.703,751250.015,1,respondent_1,2024-12-11 17:20:52,Wednesday,2,17,2024-12-11
2,2,6,6,sd_last_add,2,2,751250.015,751251.718,1,respondent_1,2024-12-11 17:20:52,Wednesday,2,17,2024-12-11
3,3,3,8,sd_abs_sub,5,5,751251.734,751254.343,1,respondent_1,2024-12-11 17:20:52,Wednesday,2,17,2024-12-11
4,4,2,8,sd_abs_sub,6,6,751254.343,751258.421,1,respondent_1,2024-12-11 17:20:52,Wednesday,2,17,2024-12-11


In [8]:
answersLog_df[answersLog_df['hour'] > 23].sort_values('hour', ascending=False)

,#,num1,num2,operation,right_ans,user_ans,start_time,end_time,correct_flg,id,datetime,weekday,weekday_num,hour,date
2800,0,9,2,sd_last_add,1,1,100758.578,100760.828,1,respondent_3,2024-12-13 02:03:23,Friday,4,26,2024-12-12
2938,138,9,3,sd_abs_sub,6,6,100979.515,100980.968,1,respondent_3,2024-12-13 02:03:23,Friday,4,26,2024-12-12
2928,128,7,7,sd_last_add,4,4,100964.031,100965.562,1,respondent_3,2024-12-13 02:03:23,Friday,4,26,2024-12-12
2929,129,7,9,sd_abs_sub,2,2,100965.562,100967.140,1,respondent_3,2024-12-13 02:03:23,Friday,4,26,2024-12-12
2930,130,3,6,sd_abs_sub,3,3,100967.140,100968.562,1,respondent_3,2024-12-13 02:03:23,Friday,4,26,2024-12-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6069,69,7,9,sd_last_add,6,6,4619.343,4621.000,1,respondent_4,2024-12-18 00:18:36,Wednesday,2,24,2024-12-17
6070,70,6,8,sd_abs_sub,2,2,4621.000,4622.250,1,respondent_4,2024-12-18 00:18:36,Wednesday,2,24,2024-12-17
6071,71,5,6,sd_last_add,1,1,4622.250,4623.500,1,respondent_4,2024-12-18 00:18:36,Wednesday,2,24,2024-12-17
6072,72,3,4,sd_last_add,7,4,4623.500,4624.765,0,respondent_4,2024-12-18 00:18:36,Wednesday,2,24,2024-12-17


## Аримфметические задачи

In [9]:
df = answersLog_df
df['response_time'] = (df['end_time'] - df['start_time'])
df.head()

,#,num1,num2,operation,right_ans,user_ans,start_time,end_time,correct_flg,id,datetime,weekday,weekday_num,hour,date,response_time
0,0,7,3,sd_abs_sub,4,4,751245.656,751247.703,1,respondent_1,2024-12-11 17:20:52,Wednesday,2,17,2024-12-11,2.047
1,1,6,1,sd_abs_sub,5,5,751247.703,751250.015,1,respondent_1,2024-12-11 17:20:52,Wednesday,2,17,2024-12-11,2.312
2,2,6,6,sd_last_add,2,2,751250.015,751251.718,1,respondent_1,2024-12-11 17:20:52,Wednesday,2,17,2024-12-11,1.703
3,3,3,8,sd_abs_sub,5,5,751251.734,751254.343,1,respondent_1,2024-12-11 17:20:52,Wednesday,2,17,2024-12-11,2.609
4,4,2,8,sd_abs_sub,6,6,751254.343,751258.421,1,respondent_1,2024-12-11 17:20:52,Wednesday,2,17,2024-12-11,4.078


In [10]:
meta_df = df.groupby(['id', 'date','weekday',  'datetime', 'hour', 'weekday_num']).agg({
    'response_time': ['mean', 'std', 'median', 'max', 'min'],
    'correct_flg': 'mean'
}).reset_index()
meta_df.columns = ['_'.join(col).strip('_') for col in meta_df.columns.values]
q_df = meta_df
q_df.set_index(['id', 'date', 'weekday']).head()

datetime  hour  weekday_num  \
id           date       weekday                                            
respondent_1 2024-12-11 Wednesday 2024-12-11 17:20:52    17            2   
             2024-12-12 Thursday  2024-12-12 13:14:12    13            3   
                        Thursday  2024-12-12 21:07:06    21            3   
             2024-12-13 Friday    2024-12-13 08:20:01     8            4   
                        Friday    2024-12-13 20:13:02    20            4   

                                   response_time_mean  response_time_std  \
id           date       weekday                                            
respondent_1 2024-12-11 Wednesday            1.899995           0.697397   
             2024-12-12 Thursday             1.651020           0.597772   
                        Thursday             1.639055           0.615554   
             2024-12-13 Friday               1.620940           0.546910   
                        Friday               1.537110           0.454181   

                                   response_time_median  response_time_max  \
id           date       weekday                                              
respondent_1 2024-12-11 Wednesday                1.6715              4.515   
             2024-12-12 Thursday                 1.4295              4.438   
                        Thursday                 1.4065              4.891   
             2024-12-13 Friday                   1.4370              5.094   
                        Friday                   1.3595              3.250   

                                   response_time_min  correct_flg_mean  
id           date       weekday                                         
respondent_1 2024-12-11 Wednesday              1.062             0.980  
             2024-12-12 Thursday               1.062             0.975  
                        Thursday               0.968             0.975  
             2024-12-13 Friday                 0.953             0.990  
                        Friday                 0.984             0.965

In [11]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
for i, (name, group) in enumerate(q_df.groupby('id')):

# Создаем пустой холст с двумя графиками по вертикали
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        subplot_titles=("Response time mean", "Response time std"))

    # Первый график — answer
    fig.add_trace(
        go.Scatter(x=group['datetime'], y=group['response_time_mean'], name='response_time_mean'),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=group['datetime'], 
                y=group['response_time_mean'], 
                    mode='markers',            
                    marker=dict(opacity=0.6),
                name='response_time_mean'
                ),
        row=1, col=1
    )

    # Второй график — stress_lvl
    fig.add_trace(
        go.Scatter(x=group['datetime'], y=group['response_time_std'], name='response_time_std'),
        row=2, col=1
    )

    fig.add_trace(
        go.Scatter(x=group['datetime'], 
                y=group['response_time_std'], 
                    mode='markers',            
                    marker=dict(opacity=0.6),
            ),
        row=2, col=1
    )


    # Общий заголовок
    fig.update_layout(height=300, title_text=f'{name}. Динамика времени на ответ', showlegend=False)

    # Обновляем подписи осей
    # fig.update_yaxes(title_text="response_time_mean", row=1, col=1)
    # fig.update_yaxes(title_text="response_time_std", row=2, col=1)
    # fig.update_yaxes(title_text="Самооценка работоспособности", row=2, col=1)
    fig.update_xaxes(title_text="Время", row=2, col=1)

    fig.show()


### Корреляции

In [12]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import dcor

# Get unique IDs
ids = q_df['id'].unique()
num_ids = len(ids)

# Parameters for subplot grid
rows, cols = num_ids, 4

# Select only numeric columns for correlation
# Select only numeric columns for correlation
numeric_df = q_df.select_dtypes(include=[np.number])
numeric_columns = numeric_df.columns

# Define the rows we want to keep
selected_rows = ['weekday_num', 'hour']

# Create figure with subplots
fig = make_subplots(
    rows=rows,
    cols=cols,
    subplot_titles=[f"ID {id} - {method}" for id in ids for method in ['Pearson', 'Spearman', 'Kendall', 'DCOR']],
    horizontal_spacing=0.1,
    vertical_spacing=0.05
)

# Calculate correlations for each respondent
for i, (name, sub_df) in enumerate(q_df.groupby('id')):
    # Calculate different correlation methods
    pearson_corr = sub_df[numeric_columns].corr(method='pearson')
    spearman_corr = sub_df[numeric_columns].corr(method='spearman')
    kendall_corr = sub_df[numeric_columns].corr(method='kendall')
    dcor_corr = pd.DataFrame(index=numeric_columns, columns=numeric_columns, dtype=float)

    for col1 in numeric_columns:
        for col2 in numeric_columns:
            dcor_corr.loc[col1, col2] = dcor.distance_correlation(
                sub_df[col1].dropna(),
                sub_df[col2].dropna()
            )
    # Create heatmaps for each correlation method
    for j, (corr, method) in enumerate([
        (pearson_corr, 'Pearson'),
        (spearman_corr, 'Spearman'),
        (kendall_corr, 'Kendall'),
        (dcor_corr, 'DCOR')
    ]):
        # Create a copy of the correlation matrix
        corr_values = corr.values.copy()
        np.fill_diagonal(corr_values, None)
        
        # Select only the rows we want to keep
        corr_values = corr_values[[list(corr.index).index(row) for row in selected_rows]]
        corr_index = selected_rows
        
        heatmap = go.Heatmap(
            z=corr_values,
            x=corr.columns,
            y=corr_index,
            colorscale='RdBu',
            zmid=0,
            colorbar=dict(title="r", len=0.3),
            showscale=(i == num_ids - 1 and j == 3)  # show colorbar only on the last plot
        )
        
        # Add heatmap to the appropriate subplot
        fig.add_trace(heatmap, row=i+1, col=j+1)

# Update layout
fig.update_layout(
    height=300 * num_ids,  # Adjust height based on number of respondents
    width=900,
    title_text="Correlation Matrices by Respondent and Method",
    font=dict(size=10),
    margin=dict(l=50, r=80, t=100, b=50)
)

# Rotate x-axis labels
for i in range(1, rows * cols + 1):
    fig['layout'][f'xaxis{i}'].update(tickangle=45)

fig.show()

c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\dcor\_fast_dcov_avl.py:554: UserWarning:

Falling back to uncompiled AVL fast distance covariance terms because of TypeError exception raised: No matching definition for argument type(s) array(int64, 1d, C), array(int64, 1d, C), bool. Rembember: only floating point values can be used in the compiled implementations.

c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\dcor\_fast_dcov_avl.py:554: UserWarning:

Falling back to uncompiled AVL fast distance covariance terms because of TypeError exception raised: No matching definition for argument type(s) array(int64, 1d, C), array(int32, 1d, C), bool. Rembember: only floating point values can be used in the compiled implementations.

c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\dcor\_fast_dcov_avl.py:554: UserWarning:

Falling back to uncompiled AVL fast distance covariance terms because of TypeError exception raised: No matchi

In [14]:
from scipy.stats import wilcoxon

answersLog_df_temp = answersLog_df.sort_values(['id', 'datetime', '#'])

# Сохраним результаты для всех респондентов и всех измерений
comparison_records = []

# По каждому пользователю
for respondent, sub_df in answersLog_df_temp.groupby('id'):
    session_times = sorted(sub_df['datetime'].unique())

    # Перебираем каждое измерение как базовое
    for i, base_time in enumerate(session_times[:-1]):
        base = sub_df[sub_df['datetime'] == base_time].sort_values('#')

        for j in range(1, 2):
            if i + j >= len(session_times):
                continue
            compare_time = session_times[i + j]
            comp = sub_df[sub_df['datetime'] == compare_time].sort_values('#')

            if len(base) == len(comp):
                try:
                    stat, p = wilcoxon(base['response_time'].values, comp['response_time'].values, alternative='greater')
                except ValueError:
                    stat, p = None, None
            else:
                stat, p = None, None

            comparison_records.append({
                'id': respondent,
                'base_time': base_time,
                'compared_to': compare_time,
                'wilcoxon_statistic': stat,
                'p_value': p
            })

# Формируем финальный DataFrame
wilcoxon_pairwise_df = pd.DataFrame(comparison_records)
wilcoxon_pairwise_df = pd.merge(wilcoxon_pairwise_df, q_df, left_on=['id', 'base_time'], right_on=['id', 'datetime'] , how='left', suffixes=('', '_bt'))
wilcoxon_pairwise_df = pd.merge(wilcoxon_pairwise_df, q_df, left_on=['id', 'compared_to'], right_on=['id', 'datetime'] , how='left', suffixes=('_bt', '_ct'))

wilcoxon_pairwise_df.head()

,id,base_time,compared_to,wilcoxon_statistic,p_value,date_bt,weekday_bt,datetime_bt,hour_bt,weekday_num_bt,...,weekday_ct,datetime_ct,hour_ct,weekday_num_ct,response_time_mean_ct,response_time_std_ct,response_time_median_ct,response_time_max_ct,response_time_min_ct,correct_flg_mean_ct
0,respondent_1,2024-12-11 17:20:52,2024-12-12 13:14:12,13083.5,0.000031,2024-12-11,Wednesday,2024-12-11 17:20:52,17,2,...,Thursday,2024-12-12 13:14:12,13,3,1.651020,0.597772,1.4295,4.438,1.062,0.975
1,respondent_1,2024-12-12 13:14:12,2024-12-12 21:07:06,10620.5,0.243178,2024-12-12,Thursday,2024-12-12 13:14:12,13,3,...,Thursday,2024-12-12 21:07:06,21,3,1.639055,0.615554,1.4065,4.891,0.968,0.975
2,respondent_1,2024-12-12 21:07:06,2024-12-13 08:20:01,9953.5,0.498283,2024-12-12,Thursday,2024-12-12 21:07:06,21,3,...,Friday,2024-12-13 08:20:01,8,4,1.620940,0.546910,1.4370,5.094,0.953,0.990
3,respondent_1,2024-12-13 08:20:01,2024-12-13 20:13:02,11503.5,0.038071,2024-12-13,Friday,2024-12-13 08:20:01,8,4,...,Friday,2024-12-13 20:13:02,20,4,1.537110,0.454181,1.3595,3.250,0.984,0.965
4,respondent_1,2024-12-13 20:13:02,2024-12-14 11:56:28,11536.5,0.025564,2024-12-13,Friday,2024-12-13 20:13:02,20,4,...,Saturday,2024-12-14 11:56:28,11,5,1.458045,0.421196,1.3280,3.250,1.031,0.980


In [16]:
# wilcoxon_pairwise_df = wilcoxon_pairwise_df[['id', 'base_time', 'compared_to', 'wilcoxon_statistic', 'p_value']]
# wilcoxon_pairwise_df['date_diff'] = ((wilcoxon_pairwise_df['compared_to'] - wilcoxon_pairwise_df['base_time']).dt.total_seconds() / (60 * 60)) // 24
wilcoxon_pairwise_df['hour_diff'] = ((wilcoxon_pairwise_df['compared_to'] - wilcoxon_pairwise_df['base_time']).dt.total_seconds() / (60 * 60)) 
wilcoxon_pairwise_df['hour_d_diff'] = ((wilcoxon_pairwise_df['compared_to'] - wilcoxon_pairwise_df['base_time']).dt.total_seconds() // (60 * 60)) % 24
wilcoxon_pairwise_df['day_time'] =  wilcoxon_pairwise_df.apply(lambda row: 'mbt_mct' if row['hour_bt'] <= 14 and row['hour_ct'] <= 14
                                                               else 'mbt_ect'if row['hour_bt'] <= 14 and row['hour_ct'] > 14
                                                               else 'ebt_mct'if row['hour_bt'] > 14 and row['hour_ct'] <= 14
                                                               else 'ebt_ect'if row['hour_bt'] > 14 and row['hour_ct'] > 14
                                                               else '-'
                                                               , axis=1)
wilcoxon_pairwise_df.set_index(['id', 'base_time'])

compared_to  wilcoxon_statistic  \
id           base_time                                                     
respondent_1 2024-12-11 17:20:52 2024-12-12 13:14:12             13083.5   
             2024-12-12 13:14:12 2024-12-12 21:07:06             10620.5   
             2024-12-12 21:07:06 2024-12-13 08:20:01              9953.5   
             2024-12-13 08:20:01 2024-12-13 20:13:02             11503.5   
             2024-12-13 20:13:02 2024-12-14 11:56:28             11536.5   
             2024-12-14 11:56:28 2024-12-14 20:53:06             11059.0   
             2024-12-14 20:53:06 2024-12-15 12:06:37              7502.5   
             2024-12-15 12:06:37 2024-12-15 22:12:43             11636.0   
respondent_2 2024-12-11 16:09:52 2024-12-12 19:57:16             15822.5   
respondent_3 2024-12-11 10:55:16 2024-12-11 15:51:05             13425.5   
             2024-12-11 15:51:05 2024-12-11 22:05:25              9902.5   
             2024-12-11 22:05:25 2024-12-13 02:03:23             12071.5   
             2024-12-13 02:03:23 2024-12-13 10:32:47             10949.0   
             2024-12-13 10:32:47 2024-12-14 13:00:47              6597.0   
             2024-12-14 13:00:47 2024-12-15 01:16:30             14989.5   
respondent_4 2024-12-12 12:56:06 2024-12-12 22:33:31              8753.0   
             2024-12-12 22:33:31 2024-12-13 11:40:45             10733.0   
             2024-12-13 11:40:45 2024-12-14 01:05:55              9685.5   
             2024-12-14 01:05:55 2024-12-14 12:11:29              9526.5   
             2024-12-14 12:11:29 2024-12-15 01:29:44             10232.5   
             2024-12-15 01:29:44 2024-12-15 13:04:17              8874.0   
             2024-12-15 13:04:17 2024-12-15 18:28:17             10573.0   
             2024-12-15 18:28:17 2024-12-16 06:52:53              8754.5   
             2024-12-16 06:52:53 2024-12-16 23:57:21             11674.5   
             2024-12-16 23:57:21 2024-12-17 09:58:55              8735.5   
             2024-12-17 09:58:55 2024-12-18 00:18:36             10066.0   
             2024-12-18 00:18:36 2024-12-18 09:33:39             11531.5   
             2024-12-18 09:33:39 2024-12-24 08:34:30              8702.5   
             2024-12-24 08:34:30 2024-12-24 22:24:22              9927.0   
             2024-12-24 22:24:22 2025-01-16 10:26:25              9475.0   
             2025-01-16 10:26:25 2025-01-20 08:42:15             10049.0   
             2025-01-20 08:42:15 2025-01-20 22:38:18             12257.0   
             2025-01-20 22:38:18 2025-01-21 11:34:02              9725.0   

                                       p_value     date_bt weekday_bt  \
id           base_time                                                  
respondent_1 2024-12-11 17:20:52  3.106059e-05  2024-12-11  Wednesday   
             2024-12-12 13:14:12  2.431785e-01  2024-12-12   Thursday   
             2024-12-12 21:07:06  4.982834e-01  2024-12-12   Thursday   
             2024-12-13 08:20:01  3.807136e-02  2024-12-13     Friday   
             2024-12-13 20:13:02  2.556392e-02  2024-12-13     Friday   
             2024-12-14 11:56:28  8.638402e-02  2024-12-14   Saturday   
             2024-12-14 20:53:06  9.981836e-01  2024-12-14   Saturday   
             2024-12-15 12:06:37  6.317908e-03  2024-12-15     Sunday   
respondent_2 2024-12-11 16:09:52  9.377214e-13  2024-12-11  Wednesday   
respondent_3 2024-12-11 10:55:16  9.657953e-06  2024-12-11  Wednesday   
             2024-12-11 15:51:05  5.714140e-01  2024-12-11  Wednesday   
             2024-12-11 22:05:25  6.820587e-03  2024-12-11  Wednesday   
             2024-12-13 02:03:23  1.363335e-01  2024-12-12     Friday   
             2024-12-13 10:32:47  9.999874e-01  2024-12-13     Friday   
             2024-12-14 13:00:47  8.347710e-10  2024-12-14   Saturday   
respondent_4 2024-12-12 12:56:06  9.294279e-01  2024-12-12   Thursday   
             2024-12-12 22:33:31  2.023147e-01  2024-12-12   Thurs

In [17]:
# wilcoxon_pairwise_df[(wilcoxon_pairwise_df['id'] == 'Дария') & (wilcoxon_pairwise_df['fatigue_lvl_diff'] > 0)]

wilcoxon_pairwise_df.groupby(['id', 'day_time']).agg(
    avg_p_val = ('p_value', 'mean'),
    mrd_p_val = ('p_value', 'median'),
    min_p_val = ('p_value', 'min'),
    max_p_val = ('p_value', 'max'),
    cnt =('id', 'count')
)

avg_p_val     mrd_p_val     min_p_val     max_p_val  \
id           day_time                                                           
respondent_1 ebt_mct   3.805155e-01  2.619237e-01  3.106059e-05  9.981836e-01   
             mbt_ect   9.348794e-02  6.222769e-02  6.317908e-03  2.431785e-01   
respondent_2 ebt_ect   9.377214e-13  9.377214e-13  9.377214e-13  9.377214e-13   
respondent_3 ebt_ect   2.891173e-01  2.891173e-01  6.820587e-03  5.714140e-01   
             ebt_mct   1.363335e-01  1.363335e-01  1.363335e-01  1.363335e-01   
             mbt_ect   4.829394e-06  4.829394e-06  8.347710e-10  9.657953e-06   
             mbt_mct   9.999874e-01  9.999874e-01  9.999874e-01  9.999874e-01   
respondent_4 ebt_mct   6.311782e-01  7.095322e-01  3.532780e-02  9.430322e-01   
             mbt_ect   3.687830e-01  3.381822e-01  1.437219e-03  9.294279e-01   
             mbt_mct   7.007480e-01  7.007480e-01  4.515655e-01  9.499306e-01   

                       cnt  
id           day_time       
respondent_1 ebt_mct     4  
             mbt_ect     4  
respondent_2 ebt_ect     1  
respondent_3 ebt_ect     2  
             ebt_mct     1  
             mbt_ect     2  
             mbt_mct     1  
respondent_4 ebt_mct     8  
             mbt_ect     8  
             mbt_mct     2

In [18]:
# wilcoxon_pairwise_df[(wilcoxon_pairwise_df['id'] == 'Дария') & (wilcoxon_pairwise_df['fatigue_lvl_diff'] > 0)]
wilcoxon_pairwise_df['rt_coef'] = wilcoxon_pairwise_df['response_time_mean_ct'] / wilcoxon_pairwise_df['response_time_mean_bt']
wilcoxon_pairwise_df['crt_coef'] = wilcoxon_pairwise_df['correct_flg_mean_ct'] / wilcoxon_pairwise_df['correct_flg_mean_bt']
wilcoxon_pairwise_df['score_diff_new'] = (wilcoxon_pairwise_df['p_value'] < 0.2).astype(int)
wilcoxon_pairwise_df.groupby(['id', 'day_time', 'score_diff_new']).agg(
    cnt =('id', 'count'),
    # avg_p_val = ('p_value', 'mean'),
    # mrd_p_val = ('p_value', 'median'),
    # min_p_val = ('p_value', 'min'),
    # max_p_val = ('p_value', 'max'),
    response_time_mean_bt = ('response_time_mean_bt', 'mean'),
        response_time_mean_ct = ('response_time_mean_ct', 'mean'),
    correct_flg_mean_bt = ('correct_flg_mean_bt', 'mean'),
        correct_flg_mean_ct = ('correct_flg_mean_ct', 'mean'),
    rt_coef = ('rt_coef', 'mean'),
    crt_coef = ('crt_coef', 'mean'),
)

cnt  response_time_mean_bt  \
id           day_time score_diff_new                               
respondent_1 ebt_mct  0                 2               1.523353   
                      1                 2               1.718553   
             mbt_ect  0                 1               1.651020   
                      1                 3               1.530130   
respondent_2 ebt_ect  1                 1               2.990650   
respondent_3 ebt_ect  0                 1               1.668740   
                      1                 1               1.691570   
             ebt_mct  1                 1               1.550630   
             mbt_ect  1                 2               1.781757   
             mbt_mct  0                 1               1.522030   
respondent_4 ebt_mct  0                 7               1.486949   
                      1                 1               1.527340   
             mbt_ect  0                 5               1.499015   
                      1                 3               1.548298   
             mbt_mct  0                 2               1.502103   

                                      response_time_mean_ct  \
id           day_time score_diff_new                          
respondent_1 ebt_mct  0                            1.566173   
                      1                            1.554532   
             mbt_ect  0                            1.639055   
                      1                            1.456952   
respondent_2 ebt_ect  1                            2.247420   
respondent_3 ebt_ect  0                            1.691570   
                      1                            1.550630   
             ebt_mct  1                            1.522030   
             mbt_ect  1                            1.547420   
             mbt_mct  0                            1.651010   
respondent_4 ebt_mct  0                            1.511502   
                      1                            1.469060   
             mbt_ect  0                            1.507805   
                      1                            1.465653   
             mbt_mct  0                            1.536985   

                                      correct_flg_mean_bt  \
id           day_time score_diff_new                        
respondent_1 ebt_mct  0                          0.977500   
                      1                          0.972500   
             mbt_ect  0                          0.975000   
                      1                          0.988333   
respondent_2 ebt_ect  1                          0.985000   
respondent_3 ebt_ect  0                          0.985000   
                      1                          0.995000   
             ebt_mct  1                          1.000000   
             mbt_ect  1                          0.997500   
             mbt_mct  0                          0.990000   
respondent_4 ebt_mct  0                          0.967857   
                      1                          0.970000   
             mbt_ect  0                          0.977000   
                      1                          0.965000   
             mbt_mct  0                          0.965000   

                                      correct_flg_mean_ct   rt_coef  crt_coef  
id           day_time score_diff_new                                           
respondent_1 ebt_mct  0                            0.9925  1.031328  1.015345  
                      1                            0.9775  0.908761  1.005221  
             mbt_ect  0                            0.9750  0.992753  1.000000  
                      1                            0.9700  0.952425  0.981532  
respondent_2 ebt_ect  1                            0.9700  0.751482  0.984772  
respondent_3 ebt_ect  0                            0.9950  1.013681  1.010152  
                      1                            1.0000  0.916681  1.005025  
             ebt_mct  1                            

In [19]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression
import numpy as np
from scipy import stats
from itertools import product


# Get unique IDs for coloring
ids = q_df['id'].unique()
num_ids = len(ids)

# Select only numeric columns for correlation
metrics = ['response_time_mean', 'response_time_median', 'response_time_std', 'correct_flg_mean']
base = ['weekday_num', 'hour']
# Create a color palette for respondents
colors = [
    '#FF0000',  # Bright Red
    '#00FF00',  # Bright Green
    '#0000FF',  # Bright Blue
    '#FF00FF',  # Magenta
    '#00FFFF',  # Cyan
    '#FFFF00',  # Yellow
    '#FFA500',  # Orange
    '#800080',  # Purple
    '#008000',  # Green
    '#000080',  # Navy
    '#800000',  # Maroon
    '#008080'   # Teal
]

# Calculate number of rows and columns for subplots
n_metrics = len(metrics)
cols = 2
rows = n_metrics

# Create figure with subplots
fig = make_subplots(
    rows=rows,
    cols=cols,
    subplot_titles=[f"{time} vs {metric})" for (time, metric) in product(metrics, base)],
    horizontal_spacing=0.1,
    vertical_spacing=0.1
)

# Create scatter plots for each metric
for i, (time, metric) in enumerate(product(metrics, base)):
    row = i // cols + 1
    col = i % cols + 1
    
    # Add scatter plot for each respondent
    for j, respondent_id in enumerate(ids):
        # print(metric, respondent_id)

        respondent_data = q_df[q_df['id'] == respondent_id].sort_values('datetime')

        # Calculate correlations
        pearson_corr = respondent_data[metric].corr(respondent_data[time], method='pearson')
        spearman_corr = respondent_data[metric].corr(respondent_data[time], method='spearman')
        kendall_corr = respondent_data[metric].corr(respondent_data[time], method='kendall')
        dcor_corr = dcor.distance_correlation(
                    respondent_data[metric].dropna().astype(float),
                    respondent_data[time].dropna().astype(float))
        scatter = go.Scatter(
            x=respondent_data[metric],
            y=respondent_data[time],
            mode='markers',
            name=f'ID: {respondent_id}',
            marker=dict(
                color=colors[j % len(colors)],
                size=8,
                opacity=0.8
            ),
            showlegend=(i == 0),
            legendgroup=f'{respondent_id}',
            hovertemplate=(
                f"ID: {respondent_id}<br>" +
                "DateTime: %{customdata}<br>" +
                f"{metric}: %{{x:.2f}}<br>" +
                "Score: %{y:.2f}<br>" +
                f"Pearson r: {pearson_corr:.3f}<br>" +
                f"Spearman r: {spearman_corr:.3f}<br>" +
                f"Kendall τ: {kendall_corr:.3f}<br>" +
                f"DCOR: {dcor_corr:.3f}"
            ),
            customdata=respondent_data['datetime']
        )
        fig.add_trace(scatter, row=row, col=col)

        # Add trendline
        x_vals = respondent_data[metric].values.reshape(-1, 1)
        y_vals = respondent_data[time].values

        if len(x_vals) > 1:
            model = LinearRegression().fit(x_vals, y_vals)
            x_range = np.linspace(x_vals.min(), x_vals.max(), 100).reshape(-1, 1)
            y_pred = model.predict(x_range)

            # Create annotation text with correlation values
            annotation_text = (
                f"ID: {respondent_id}<br>" +
                f"Pearson r: {pearson_corr:.3f}<br>" +
                f"Spearman r: {spearman_corr:.3f}<br>" +
                f"Kendall τ: {kendall_corr:.3f}<br>" +
                f"DCOR: {dcor_corr:.3f}"
            )

            fig.add_trace(
                go.Scatter(
                    x=x_range.flatten(),
                    y=y_pred,
                    mode='lines',
                    name=f'Trend {respondent_id}',
                    line=dict(
                        color=colors[j % len(colors)],
                        width=2,
                        dash='dash'
                    ),
                    showlegend=False,
                    legendgroup=f'{respondent_id}',
                    hovertemplate=annotation_text
                ),
                row=row, col=col
            )

# Update layout
fig.update_layout(
    height=300 * rows,
    width=1200,
    title_text="Metrics vs Score by Respondent",
    font=dict(size=12),
    margin=dict(l=50, r=50, t=100, b=50),
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    )
)

# Update axes labels
for i in range(0, rows * cols ):
    if i > len(metrics):
        continue
    # print(base[i % 2])
    # print(metrics[i // cols +1])
    fig['layout'][f'xaxis{i // cols +1}'].update(title=metrics[i // cols])
    fig['layout'][f'yaxis{i % cols +1}'].update(title=base[i % cols])

fig.show()

In [22]:
import pandas as pd
import plotly.express as px


# Создадим столбцы с индивидуальными корреляциями r и r² для каждого пользователя
correlation_data = wilcoxon_pairwise_df.groupby('id').apply(
    lambda g: pd.Series({
        'corr': g['p_value'].corr(g['hour_d_diff']),
        'r_squared': g['p_value'].corr(g['hour_d_diff']) ** 2
    })
).reset_index()
# Объединяем обратно в основной датафрейм
w_df = wilcoxon_pairwise_df.merge(correlation_data, on='id', how='left')
w_df = w_df[w_df['day_time'] == 'mbt_ect']
# Добавляем аннотацию
w_df['hover_text'] = (
    'ID: ' + w_df['id'].astype(str) +
    '<br>Wilcoxon: ' + w_df['p_value'].round(2).astype(str) +
    # '<br>Score Δ: ' + wilcoxon_pairwise_df['fatigue_lvl_diff'].round(2).astype(str) +
    '<br>r: ' + w_df['corr'].round(2).astype(str) +
    '<br>r²: ' + w_df['r_squared'].round(2).astype(str)
)

# Визуализация

fig = px.scatter(
    w_df,
    x='hour_d_diff',
    
    y='p_value',
    color='id',
    hover_name='id',
    hover_data={'id': False, 'p_value': False, 'hour_d_diff': False, 'hover_text': True},
    # title='Wilcoxon vs Score Change по респондентам с индивидуальными r и r²',
    labels={
        'hour_d_diff': 'Изменение hour',
        'p_value': 'Статистика Wilcoxon',
        'id': 'Респондент'
    },
    trendline='ols'
)

fig.update_traces(marker=dict(size=8, opacity=0.7), selector=dict(mode='markers'))
fig.update_layout(legend_title_text='Респонденты')
fig.show()


c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning:

Degrees of freedom <= 0 for slice

c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning:

divide by zero encountered in divide

c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning:

invalid value encountered in multiply



### Критерий недельный

In [23]:
# смотрим на разницу между утрами
import pandas as pd
from scipy.stats import wilcoxon

# Загрузим правильный файл
df_new = answersLog_df.copy()
print(df_new.dtypes)
# df_new['date'] =  df_new[['datetime', 'hour', 'date']].apply(lambda row: (row['datetime'] - pd.Timedelta(days=1)).date() if row['hour'] > 23 else row['date'], axis=1)
df_new['week_start'] = df_new['date'].apply(lambda r: pd.to_datetime(r).to_period('W').start_time)

# df_new['rank_week'] = df_new.groupby(['id', 'week_start'])['datetime'].rank(method='dense')
df_new['rank'] = df_new.groupby(['id', 'date'])['hour'].rank(method='dense')
df_new['rank_day'] = df_new.groupby(['id'])['date'].rank(method='dense')

results = []
# Промечаем бейзлайны
for (respondent_id, date, datetime, rank, day_rank), group in df_new.groupby(['id', 'date', 'datetime', 'rank', 'rank_day']):
    # print(respondent_id, str(datetime), group['rank'].mean())
    if day_rank == 1 or rank != 1:
        continue
    baseline_session = df_new[(df_new['rank'] == 1) & (df_new['rank_day'] == day_rank - 1) & (df_new['id'] == respondent_id)]
    print(baseline_session.shape, group.shape)
    print(respondent_id, baseline_session.datetime.max(), group.datetime.max())
    if baseline_session.shape[0] == 0:
        continue
    stat_g, p_g = wilcoxon(baseline_session['response_time'], group['response_time'],  alternative='greater')
    stat_l, p_l = wilcoxon(baseline_session['response_time'],group['response_time'], alternative='less')
    stat, p = wilcoxon(baseline_session['response_time'],group['response_time'])

    results.append({
        'id': respondent_id,
        'base_time': baseline_session.datetime.max(),
        'session_time': datetime,
        'wilcoxon_statistic_g': stat_g,
        'p_value_g': p_g,
        'wilcoxon_statistic_l': stat_l,
        'p_value_l': p_l,
        'wilcoxon_statistic': stat,
        'p_value': p,
        'rank' : rank,
        'day_rank' : day_rank,
        
        # 'fatigue_lvl': gr
    })

# # Финальный результат
wilcoxon_result_df = pd.DataFrame(results)

wilcoxon_result_df[(wilcoxon_result_df['p_value'] < 0.05) | (wilcoxon_result_df['p_value_l'] < 0.05) | (wilcoxon_result_df['p_value_g'] < 0.05)]

#                         int64
num1                      int64
num2                      int64
operation                object
right_ans                 int64
user_ans                  int64
start_time              float64
end_time                float64
correct_flg               int64
id                       object
datetime         datetime64[ns]
weekday                  object
weekday_num               int32
hour                      int64
date                     object
response_time           float64
dtype: object
(200, 19) (200, 19)
respondent_1 2024-12-11 17:20:52 2024-12-12 13:14:12
(200, 19) (200, 19)
respondent_1 2024-12-12 13:14:12 2024-12-13 08:20:01
(200, 19) (200, 19)
respondent_1 2024-12-13 08:20:01 2024-12-14 11:56:28
(200, 19) (200, 19)
respondent_1 2024-12-14 11:56:28 2024-12-15 12:06:37
(200, 19) (200, 19)
respondent_2 2024-12-11 16:09:52 2024-12-12 19:57:16
(200, 19) (200, 19)
respondent_3 2024-12-11 10:55:16 2024-12-13 02:03:23
(200, 19) (200, 19)
respondent_3 202

,id,base_time,session_time,wilcoxon_statistic_g,p_value_g,wilcoxon_statistic_l,p_value_l,wilcoxon_statistic,p_value,rank,day_rank
0,respondent_1,2024-12-11 17:20:52,2024-12-12 13:14:12,13083.5,3.106059e-05,13083.5,0.999969,6617.5,6.212119e-05,1.0,2.0
2,respondent_1,2024-12-13 08:20:01,2024-12-14 11:56:28,13304.5,3.577842e-05,13304.5,0.999964,6795.5,7.155685e-05,1.0,4.0
3,respondent_1,2024-12-14 11:56:28,2024-12-15 12:06:37,8460.5,9.664602e-01,8460.5,0.033540,8460.5,6.707960e-02,1.0,5.0
4,respondent_2,2024-12-11 16:09:52,2024-12-12 19:57:16,15822.5,9.377214e-13,15822.5,1.000000,4277.5,1.875443e-12,1.0,2.0
5,respondent_3,2024-12-11 10:55:16,2024-12-13 02:03:23,15578.5,2.266886e-12,15578.5,1.000000,4321.5,4.533773e-12,1.0,2.0
7,respondent_3,2024-12-13 10:32:47,2024-12-14 13:00:47,6597.0,9.999874e-01,6597.0,0.000013,6597.0,2.516863e-05,1.0,4.0
17,respondent_4,2025-01-20 08:42:15,2025-01-21 11:34:02,11977.5,2.732683e-03,11977.5,0.997267,7525.5,5.465367e-03,1.0,11.0


In [24]:
# разница между вечерами
import pandas as pd
from scipy.stats import wilcoxon

# Загрузим правильный файл
df_new = answersLog_df.copy()
print(df_new.dtypes)
# df_new['date'] =  df_new[['datetime', 'hour', 'date']].apply(lambda row: (row['datetime'] - pd.Timedelta(days=1)).date() if row['hour'] > 23 else row['date'], axis=1)
df_new['week_start'] = df_new['date'].apply(lambda r: pd.to_datetime(r).to_period('W').start_time)

# df_new['rank_week'] = df_new.groupby(['id', 'week_start'])['datetime'].rank(method='dense')
df_new['rank'] = df_new.groupby(['id', 'date'])['hour'].rank(method='dense')
df_new['rank_day'] = df_new.groupby(['id'])['date'].rank(method='dense')
# Сессии — уникальные сочетания id + datetime
# sessions = df_new[['id', 'datetime', 'date']].drop_duplicates()

results = []
# Промечаем бейзлайны
for (respondent_id, date, datetime, rank, day_rank), group in df_new.groupby(['id', 'date', 'datetime', 'rank', 'rank_day']):
    # print(respondent_id, str(datetime), group['rank'].mean())
    if day_rank == 1 or rank != 2:
        continue
    baseline_session = df_new[(df_new['rank'] == 2) & (df_new['rank_day'] == day_rank - 1) & (df_new['id'] == respondent_id)]
    print(baseline_session.shape, group.shape)
    print(respondent_id, baseline_session.datetime.max(), group.datetime.max())
    if baseline_session.shape[0] == 0:
        continue
    stat_g, p_g = wilcoxon(baseline_session['response_time'], group['response_time'],  alternative='greater')
    stat_l, p_l = wilcoxon(baseline_session['response_time'],group['response_time'], alternative='less')
    stat, p = wilcoxon(baseline_session['response_time'],group['response_time'])

    results.append({
        'id': respondent_id,
        'base_time': baseline_session.datetime.max(),
        'session_time': datetime,
        'wilcoxon_statistic_g': stat_g,
        'p_value_g': p_g,
        'wilcoxon_statistic_l': stat_l,
        'p_value_l': p_l,
        'wilcoxon_statistic': stat,
        'p_value': p,
        
        # 'fatigue_lvl': gr
    })

# # Финальный результат
wilcoxon_result_df = pd.DataFrame(results)

wilcoxon_result_df[(wilcoxon_result_df['p_value'] < 0.05) | (wilcoxon_result_df['p_value_l'] < 0.05) | (wilcoxon_result_df['p_value_g'] < 0.05)]

#                         int64
num1                      int64
num2                      int64
operation                object
right_ans                 int64
user_ans                  int64
start_time              float64
end_time                float64
correct_flg               int64
id                       object
datetime         datetime64[ns]
weekday                  object
weekday_num               int32
hour                      int64
date                     object
response_time           float64
dtype: object
(0, 19) (200, 19)
respondent_1 NaT 2024-12-12 21:07:06
(200, 19) (200, 19)
respondent_1 2024-12-12 21:07:06 2024-12-13 20:13:02
(200, 19) (200, 19)
respondent_1 2024-12-13 20:13:02 2024-12-14 20:53:06
(200, 19) (200, 19)
respondent_1 2024-12-14 20:53:06 2024-12-15 22:12:43
(0, 19) (200, 19)
respondent_3 NaT 2024-12-15 01:16:30
(200, 19) (200, 19)
respondent_4 2024-12-12 22:33:31 2024-12-14 01:05:55
(200, 19) (200, 19)
respondent_4 2024-12-14 01:05:55 2024-12-15 01:29:44

,id,base_time,session_time,wilcoxon_statistic_g,p_value_g,wilcoxon_statistic_l,p_value_l,wilcoxon_statistic,p_value
1,respondent_1,2024-12-13 20:13:02,2024-12-14 20:53:06,12466.0,0.00099,12466.0,0.99901,7434.0,0.001981


In [25]:
#  один день  - ~4 измерения
import pandas as pd
from scipy.stats import wilcoxon

# Загрузим правильный файл
df_new = answersLog_df.copy()
# df_new['date'] =  df_new[['datetime', 'hour', 'date']].apply(lambda row: (row['datetime'] - pd.Timedelta(days=1)).date() if row['hour'] > 23 else row['date'], axis=1)
df_new['week_start'] = df_new['date'].apply(lambda r: pd.to_datetime(r).to_period('W').start_time)

# df_new['rank_week'] = df_new.groupby(['id', 'week_start'])['datetime'].rank(method='dense')
df_new['rank'] = df_new.groupby(['id', 'date'])['datetime'].rank(method='dense')
df_new['rank_day'] = df_new.groupby(['id'])['date'].rank(method='dense')

results = []
# Промечаем бейзлайны
for (respondent_id, date, datetime, rank, day_rank), group in df_new.groupby(['id', 'date', 'datetime', 'rank', 'rank_day']):
    # print(respondent_id, str(datetime), group['rank'].mean())
    # if rank != 1:
    #     continue
    for i in range(int(rank) + 1,5):
        session = df_new[(df_new['rank'] == i) & (df_new['rank_day'] == day_rank) & (df_new['id'] == respondent_id)]
        if session.shape[0] == 0:
            continue
        stat_g, p_g = wilcoxon(session['response_time'], group['response_time'],  alternative='greater')
        stat_l, p_l = wilcoxon(session['response_time'], group['response_time'], alternative='less')
        stat, p = wilcoxon(session['response_time'], group['response_time'])

        results.append({
            'id': respondent_id,
            'base_time': datetime,
            'session_time': session.datetime.max(),
            'wilcoxon_statistic_g': stat_g,
            'p_value_g': p_g,
            'wilcoxon_statistic_l': stat_l,
            'p_value_l': p_l,
            'wilcoxon_statistic': stat,
            'p_value': p,
            'rank_bt' : rank,
            'rank_st' : session['rank'].max(),
            'day_rank' : day_rank,
            
            # 'fatigue_lvl': gr
        })
        
# # Финальный результат
wilcoxon_result_df = pd.DataFrame(results)

merge_1 = q_df
wilcoxon_result_df = pd.merge(wilcoxon_result_df, merge_1, left_on=['id', 'base_time'], right_on=['id', 'datetime'] , how='left')


wilcoxon_result_df = pd.merge(wilcoxon_result_df, merge_1, left_on=['id', 'session_time'], right_on=['id', 'datetime'] , how='left', suffixes=('_bt', '_st'))


# wilcoxon_pairwise_df[(wilcoxon_pairwise_df['id'] == 'Дария') & (wilcoxon_pairwise_df['fatigue_lvl_diff'] > 0)]
wilcoxon_result_df['score_diff_new'] = (wilcoxon_result_df['p_value'] < 0.2).astype(int)
wilcoxon_result_df['rt_coef'] = wilcoxon_result_df['response_time_mean_st'] /  wilcoxon_result_df['response_time_mean_bt']
wilcoxon_result_df['rt_std_coef'] = wilcoxon_result_df['response_time_std_st'] /  wilcoxon_result_df['response_time_std_bt']
wilcoxon_result_df['crct_coef'] = wilcoxon_result_df['correct_flg_mean_st'] /  wilcoxon_result_df['correct_flg_mean_bt']
# display(wilcoxon_result_df.groupby(['fatigue_lvl_bt', 'fatigue_lvl_st']).agg(
#     cnt =('id', 'count'),
#     avg_p_val = ('p_value', 'mean'),
#     p_value_g = ('p_value_g', 'mean'),
#     p_value_l = ('p_value_l', 'mean'),

#     rt_coef = ('rt_coef', 'mean'),
#     rt_std_coef = ('rt_std_coef', 'mean'),
#     crct_coef = ('crct_coef', 'mean'),
# ))

wilcoxon_result_df.groupby(['rank_bt', 'rank_st']).agg(
    cnt =('id', 'count'),
    avg_p_val = ('p_value', 'mean'),
    p_value_g = ('p_value_g', 'mean'),
    p_value_l = ('p_value_l', 'mean'),

    rt_coef = ('rt_coef', 'mean'),
    rt_std_coef = ('rt_std_coef', 'mean'),
    crct_coef = ('crct_coef', 'mean'),
)

cnt  avg_p_val  p_value_g  p_value_l   rt_coef  rt_std_coef  \
rank_bt rank_st                                                                
1.0     2.0       14   0.312018   0.762555   0.237445  0.961418     0.964706   
        3.0        1   0.000004   0.999998   0.000002  0.884479     0.992296   
2.0     3.0        1   0.857172   0.428586   0.571414  1.013681     1.222314   

                 crct_coef  
rank_bt rank_st             
1.0     2.0       0.992485  
        3.0       1.000000  
2.0     3.0       1.010152

In [26]:
#  один день  - ~4 измерения
import pandas as pd
from scipy.stats import wilcoxon

# Загрузим правильный файл
df_new = answersLog_df.copy()
# df_new['date'] =  df_new[['datetime', 'hour', 'date']].apply(lambda row: (row['datetime'] - pd.Timedelta(days=1)).date() if row['hour'] > 23 else row['date'], axis=1)
df_new['week_start'] = df_new['date'].apply(lambda r: pd.to_datetime(r).to_period('W').start_time)

# df_new['rank_week'] = df_new.groupby(['id', 'week_start'])['datetime'].rank(method='dense')
df_new['rank'] = df_new.groupby(['id', 'date'])['datetime'].rank(method='dense')
df_new['rank_day'] = df_new.groupby(['id'])['date'].rank(method='dense')

results = []
# Промечаем бейзлайны
for (respondent_id, date, datetime, rank, day_rank), group in df_new.groupby(['id', 'date', 'datetime', 'rank', 'rank_day']):
    # print(respondent_id, str(datetime), group['rank'].mean())
    # if rank != 1:
    #     continue
    for i in range(int(rank) + 1,5):
        session = df_new[(df_new['rank'] == i) & (df_new['rank_day'] == day_rank) & (df_new['id'] == respondent_id)]
        if session.shape[0] == 0:
            continue
        stat_g, p_g = wilcoxon(session['response_time'], group['response_time'],  alternative='greater')
        stat_l, p_l = wilcoxon(session['response_time'], group['response_time'], alternative='less')
        stat, p = wilcoxon(session['response_time'], group['response_time'])

        results.append({
            'id': respondent_id,
            'base_time': datetime,
            'session_time': session.datetime.max(),
            'wilcoxon_statistic_g': stat_g,
            'p_value_g': p_g,
            'wilcoxon_statistic_l': stat_l,
            'p_value_l': p_l,
            'wilcoxon_statistic': stat,
            'p_value': p,
            'rank_bt' : rank,
            'rank_st' : session['rank'].max(),
            'day_rank' : day_rank,
            
            # 'fatigue_lvl': gr
        })
        
# # Финальный результат
wilcoxon_result_df = pd.DataFrame(results)

merge_1 = q_df
wilcoxon_result_df = pd.merge(wilcoxon_result_df, merge_1, left_on=['id', 'base_time'], right_on=['id', 'datetime'] , how='left')


wilcoxon_result_df = pd.merge(wilcoxon_result_df, merge_1, left_on=['id', 'session_time'], right_on=['id', 'datetime'] , how='left', suffixes=('_bt', '_st'))


# wilcoxon_pairwise_df[(wilcoxon_pairwise_df['id'] == 'Дария') & (wilcoxon_pairwise_df['fatigue_lvl_diff'] > 0)]
wilcoxon_result_df['score_diff_new'] = (wilcoxon_result_df['p_value'] < 0.2).astype(int)
wilcoxon_result_df['rt_coef'] = wilcoxon_result_df['response_time_mean_st'] /  wilcoxon_result_df['response_time_mean_bt']
wilcoxon_result_df['rt_std_coef'] = wilcoxon_result_df['response_time_std_st'] /  wilcoxon_result_df['response_time_std_bt']
wilcoxon_result_df['crct_coef'] = wilcoxon_result_df['correct_flg_mean_st'] /  wilcoxon_result_df['correct_flg_mean_bt']
display(wilcoxon_result_df)
# display(wilcoxon_result_df.groupby(['fatigue_lvl_bt', 'fatigue_lvl_st']).agg(
#     cnt =('id', 'count'),
#     avg_p_val = ('p_value', 'mean'),
#     p_value_g = ('p_value_g', 'mean'),
#     p_value_l = ('p_value_l', 'mean'),

#     rt_coef = ('rt_coef', 'mean'),
#     rt_std_coef = ('rt_std_coef', 'mean'),
#     crct_coef = ('crct_coef', 'mean'),
# ))

wilcoxon_result_df.groupby(['id','rank_bt', 'rank_st']).agg(
    cnt =('id', 'count'),
    avg_p_val = ('p_value', 'mean'),
    p_value_g = ('p_value_g', 'mean'),
    p_value_l = ('p_value_l', 'mean'),
    response_time_std_bt = ('response_time_mean_bt', 'mean'),
    response_time_std_st = ('response_time_mean_st', 'mean'),

    rt_coef = ('rt_coef', 'mean'),
    rt_std_coef = ('rt_std_coef', 'mean'),
    crct_coef = ('crct_coef', 'mean'),
)

,id,base_time,session_time,wilcoxon_statistic_g,p_value_g,wilcoxon_statistic_l,p_value_l,wilcoxon_statistic,p_value,rank_bt,...,response_time_mean_st,response_time_std_st,response_time_median_st,response_time_max_st,response_time_min_st,correct_flg_mean_st,score_diff_new,rt_coef,rt_std_coef,crct_coef
0,respondent_1,2024-12-12 13:14:12,2024-12-12 21:07:06,9479.5,0.756822,9479.5,2.431785e-01,9479.5,4.863570e-01,1.0,...,1.639055,0.615554,1.4065,4.891,0.968,0.975,0,0.992753,1.029746,1.000000
1,respondent_1,2024-12-13 08:20:01,2024-12-13 20:13:02,8596.5,0.961929,8596.5,3.807136e-02,8596.5,7.614272e-02,1.0,...,1.537110,0.454181,1.3595,3.250,0.984,0.965,1,0.948283,0.830449,0.974747
2,respondent_1,2024-12-14 11:56:28,2024-12-14 20:53:06,8841.0,0.913616,8841.0,8.638402e-02,8841.0,1.727680e-01,1.0,...,1.407650,0.423153,1.2650,4.484,0.875,0.980,1,0.965437,1.004648,1.000000
3,respondent_1,2024-12-15 12:06:37,2024-12-15 22:12:43,7670.0,0.993682,7670.0,6.317908e-03,7670.0,1.263582e-02,1.0,...,1.426095,0.384864,1.2970,3.063,1.000,0.965,1,0.943556,0.827390,0.969849
4,respondent_3,2024-12-11 10:55:16,2024-12-11 15:51:05,6474.5,0.999990,6474.5,9.657953e-06,6474.5,1.931591e-05,1.0,...,1.668740,0.531536,1.5160,4.703,0.781,0.985,1,0.872542,0.811817,0.989950
5,respondent_3,2024-12-11 10:55:16,2024-12-11 22:05:25,6249.5,0.999998,6249.5,1.765296e-06,6249.5,3.530593e-06,1.0,...,1.691570,0.649704,1.5310,5.718,0.953,0.995,1,0.884479,0.992296,1.000000
6,respondent_3,2024-12-11 15:51:05,2024-12-11 22:05:25,10197.5,0.428586,10197.5,5.714140e-01,9902.5,8.571720e-01,2.0,...,1.691570,0.649704,1.5310,5.718,0.953,0.995,0,1.013681,1.222314,1.010152
7,respondent_3,2024-12-14 13:00:47,2024-12-15 01:16:30,5110.5,1.000000,5110.5,8.347710e-10,5110.5,1.669542e-09,1.0,...,1.426100,0.414898,1.3435,3.937,0.765,0.995,1,0.863774,0.930349,0.995000
8,respondent_4,2024-12-12 12:56:06,2024-12-12 22:33:31,11147.0,0.070572,11147.0,9.294279e-01,8753.0,1.411442e-01,1.0,...,1.540920,0.578648,1.3515,5.313,0.985,0.965,1,1.062645,1.394651,0.965000
9,respondent_4,2024-12-13 11:40:45,2024-12-14 01:05:55,10015.5,0.419027,10015.5,5.809727e-01,9685.5,8.380546e-01,1.0,...,1.513275,0.510082,1.3435,4.547,0.922,0.970,0,1.036664,1.384297,1.000000


cnt  avg_p_val  p_value_g  p_value_l  \
id           rank_bt rank_st                                         
respondent_1 1.0     2.0        4   0.186976   0.906512   0.093488   
respondent_3 1.0     2.0        2   0.000010   0.999995   0.000005   
                     3.0        1   0.000004   0.999998   0.000002   
             2.0     3.0        1   0.857172   0.428586   0.571414   
respondent_4 1.0     2.0        8   0.452540   0.631217   0.368783   

                              response_time_std_bt  response_time_std_st  \
id           rank_bt rank_st                                               
respondent_1 1.0     2.0                  1.560353              1.502478   
respondent_3 1.0     2.0                  1.781757              1.547420   
                     3.0                  1.912505              1.691570   
             2.0     3.0                  1.668740              1.691570   
respondent_4 1.0     2.0                  1.517496              1.491998   

                               rt_coef  rt_std_coef  crct_coef  
id           rank_bt rank_st                                    
respondent_1 1.0     2.0      0.962507     0.923058   0.986149  
respondent_3 1.0     2.0      0.868158     0.871083   0.992475  
                     3.0      0.884479     0.992296   1.000000  
             2.0     3.0      1.013681     1.222314   1.010152  
respondent_4 1.0     2.0      0.984189     1.008936   0.995655

# Выводы

Первоначальная гипотеза
- между показателями утра и вечера будет разница вне зависимости от начального состояния
- случайная смена вида операций поможет добавить когнитивную нагрузку

Выводы
- есть стабильность - утро от утра не сильно отличается (с вечером та же фигня)
- но он говорит что есть разница в те дни, когда по отзывам пользователей не было значительной когнитивной нагрузки (отмечали физическую или небольшую когнитивную нагрузку)